# NNodely Documentation - Inference

Here are shown all the modalities in which you can make inference with the model.

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

## Simple inference

To make inference, just call the nnodely framework class by passing the dictionary of inputs. The model will execute a forward pass and return the outputs of the network.

It is mandatory to call the inference with at least 1 possible prediction window for each declared inputs. the framework will predict the maximum possible window given the data

In [ ]:
## Model definition
x = Input('x')
F = Input('F')

px = Parameter('px', tw=0.5, values=[[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.]])
pf = Parameter('pf', sw=1, values=[[1.]])
next_x = Fir(W=px)(x.tw(0.5))+Fir(W=pf)(F.last())

out = Output('next_x', next_x)

model = Modely()
model.addModel('model',[out])
model.neuralizeModel(0.1)

In [ ]:
## Inference
results = model(inputs={'F':[[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]})
print(results)

In [ ]:
results = model(inputs={'F':[[5],[4],[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13]]})
print(results)

## Inference with a sampled data

Using the option sampled=True the inference window is left to the user to define by passing every sampling input.

In [ ]:
results = model(inputs={'F':[[5],[2]],'x':[[1,2,3,4,5,6,7,8,9,10,11],[12,13,14,15,16,17,18,19,20,21,22]]}, sampled=True)
print(results)